# Covid vs India 
> One of the biggest question I have been wondering is , with every adult allowed now to register, Where are vaccines slots available in india ? How can I make this decision programmitcally because trying to find information on government website is rather "sisyphus task".
So with rather self involed goal in mind, We would using publicly avaialbe data try to answer following questions
- toc: false
- branch: master
- badges: false
- comments: true
- categories: [dfs, jupyter, python]
- image: images/statistics.png
- hide: false

1. what's ground reality for Key states like Delhi, Maharastra , Tamil Nadu
2. What's ground reality for populus states like UP
3. how does it fare around 
2. Latest vaccine slots per district and State in India
2. Top 5 States Running behind on schedule
3. Top 5 States Running ahead of everyone else.
4. When can I latest find slots in Delhi, Bombay, Chennai and Bangalore for people between 18-45


"One algorithm per week with its real world use case. We are pickn"



Thanks to https://github.com/bhattbhavesh91/cowin-vaccination-slot-availability/blob/main/cowin-api-availability.ipynb for doing actual work, I picked up loads of stuff from there.


In [250]:
!{sys.executable} -m pip install --user install requests
!{sys.executable} -m pip install --user install altair
!{sys.executable} -m pip install --user install pandas
!{sys.executable} -m pip install --user install geopandas
#hide

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


Get All libraries in place

In [251]:
import geopandas as gpd
import pandas as pd
import requests
import json
from collections import defaultdict
from dataclasses import dataclass, asdict
import datetime
from typing import List
import uuid

Lets make a data class to store our geographical and vaccination data

In [252]:
@dataclass
class District:
    district_id:int = None
    district_name:str = None
    state_id:int = None


@dataclass
class Session:
    session_uuid:str = None
    date:datetime.datetime = None
    query_date:datetime.datetime= None
    available_capacity:int = None
    min_age_limit:int = None
    vaccine:str = None
    center_id:str = None
    district_id:str = None
        
@dataclass
class Center:
    center_uuid:str = None
    center_id:int = None
    center_name:str = None
    state_name:str = None
    district_name:str = None
    block_name:str = None
    pincode:str = None
    lat:int = None
    lng:int = None
    from_hour:datetime.datetime = None
    to_hour:datetime.datetime = None
    fee_type:str = None
    district_id:str = None

@dataclass
class NoSlotAvailable:
    district_id:str = None
    date:datetime.datetime = None

Lets call the API to get the populate geographical data

In [253]:
MOZILLA_HEADER = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
GET_DISTRICT_DATA_API_URL = "https://cdn-api.co-vin.in/api/v2/admin/location/districts/{}"
GET_APOINTMENT_DATA_API_URL = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id={}&date={}"

DISTRICTS = defaultdict(District)
CENTERS = defaultdict(Center)
SESSIONS = defaultdict(Session)
NO_SLOT_AVAILABLE = []

MAX_NUMBER_OF_STATES = 40

for state_code in range(1, MAX_NUMBER_OF_STATES):
    headers = {'User-Agent': MOZILLA_HEADER}
    response = requests.get(GET_DISTRICT_DATA_API_URL.format(state_code), headers=headers)
    districts_data = json.loads(response.content)
    for district in districts_data['districts']:
        district_name = district['district_name']
        district_id = district['district_id']
        district = District(district_name = district_name, district_id = district_id , state_id=state_code) 
        DISTRICTS[district_id] = district

In [254]:
DISTRICT_DF = pd.DataFrame.from_dict([asdict(district) for district in DISTRICTS.values()])

Lets now call actual api to get slots 

In [255]:
MAX_DAYS = 7

#This step takes loads of time
def get_days_in_future_from_today():
    base = datetime.datetime.today()
    date_list = [base + datetime.timedelta(days=x) for x in range(MAX_DAYS)]
    return [x.strftime("%d-%m-%Y") for x in date_list]

for district_id in DISTRICTS.keys():
    for slot_date in get_data_fiften_days_in_future_from_today():
        URL = GET_APOINTMENT_DATA_API_URL.format(district_id, slot_date)
        response = requests.get(URL)
        if response.ok:
            resp_json = response.json()
            if resp_json["centers"]:
                for center in resp_json["centers"]:
                    center_uuid = str(uuid.uuid4())
                    center_id = center["center_id"]
                    center_name = center["name"]
                    CENTERS[center_uuid] = Center(center_uuid=center_uuid,
                                                center_id=center_id,
                                                center_name=center_name,
                                                lat=center["lat"],
                                                lng=center["long"],
                                                from_hour=center["from"],
                                                to_hour=center["to"],
                                                district_id=district_id,
                                                state_name=center["state_name"],
                                                district_name=center["district_name"],
                                                block_name=center["block_name"],
                                                pincode=center["pincode"],
                                                fee_type=center["fee_type"])
                    for session in center["sessions"]:
                        session_id = session["session_id"]
                        SESSIONS[session_id] = Session(session_uuid=session_id,
                                                       date=session["date"],
                                                       query_date=slot_date,
                                                       available_capacity=session["available_capacity"],
                                                       min_age_limit=session["min_age_limit"],
                                                       vaccine=session["vaccine"],
                                                       district_id=district_id,
                                                       center_id=center_id)
            else:
                NO_SLOT_AVAILABLE.append(NoSlotAvailable(district_id=district_id, date=slot_date))
#                 print("No slot on {} in district {}".format(slot_date, district_id))

In [256]:
CENTER_DF = pd.DataFrame.from_dict([asdict(district) for district in CENTERS.values()])
SESSION_DF = pd.DataFrame.from_dict([asdict(session) for session in SESSIONS.values()])
NO_SLOT_AVAILABLE_DF = pd.DataFrame.from_dict([asdict(no_slot_available) for no_slot_available in NO_SLOT_AVAILABLE])

In [257]:
NO_SLOT_AVAILABLE_DF.head()

,district_id,date
0,2,06-05-2021
1,2,07-05-2021
2,22,01-05-2021
3,22,02-05-2021
4,22,03-05-2021


In [258]:
SESSION_DF.head()

,session_uuid,date,query_date,available_capacity,min_age_limit,vaccine,center_id,district_id
0,51738255-efad-470e-8660-e9d829d83ccb,01-05-2021,01-05-2021,50.0,45,,570779,3
1,207fd35f-888d-460d-8304-f0b99f9a13a0,03-05-2021,03-05-2021,50.0,45,,570779,3
2,88373311-b0da-40f6-bbea-ee029f86eb1c,04-05-2021,04-05-2021,50.0,45,,570779,3
3,8ef1ec99-7318-4cb6-bc66-9ed3675f9a48,05-05-2021,05-05-2021,49.0,45,,570779,3
4,bd311aec-c4cf-45f3-8f06-2dbc7b634de4,06-05-2021,06-05-2021,50.0,45,,570779,3


In [259]:
CENTER_DF.head()

,center_uuid,center_id,center_name,state_name,district_name,block_name,pincode,lat,lng,from_hour,to_hour,fee_type,district_id
0,c7dad516-fa4c-4165-a731-3cf5692f7c42,570779,BJR Hospital,Andaman and Nicobar Islands,Nicobar,Car Nicobar,744301,9.0,92.0,09:00:00,17:00:00,Free,3
1,5d109d18-c6d6-4515-b4b8-9949afb8bfd8,552108,Nancowry CHC,Andaman and Nicobar Islands,Nicobar,Nancowry,744303,7.0,93.0,09:00:00,17:00:00,Free,3
2,508df864-75dd-4e98-a359-689f73a8dc07,552109,Campbellbay PHC,Andaman and Nicobar Islands,Nicobar,Campbell Bay,744302,7.0,93.0,09:00:00,17:00:00,Free,3
3,f52f5a71-c4cc-43fc-960d-bcadd286a7ea,639986,SMC 37 WING,Andaman and Nicobar Islands,Nicobar,Car Nicobar,744301,9.0,92.0,09:00:00,18:00:00,Free,3
4,4474777c-9750-463e-bb80-f9ec292c1fba,570779,BJR Hospital,Andaman and Nicobar Islands,Nicobar,Car Nicobar,744301,0.0,0.0,09:00:00,17:00:00,Free,3


In [260]:
DISTRICT_DF.head()

,district_id,district_name,state_id
0,3,Nicobar,1
1,1,North and Middle Andaman,1
2,2,South Andaman,1
3,9,Anantapur,2
4,10,Chittoor,2


In [261]:
session_center_merged_df = pd.merge(SESSION_DF, CENTER_DF, on="center_id")
session_center_district_df = pd.merge(session_center_merged_df, DISTRICT_DF, left_on='district_id_x', right_on="district_id")

In [262]:
session_center_district_df.to_csv("vaccination_slot_data.csv")
session_center_district_df

,session_uuid,date,query_date,available_capacity,min_age_limit,vaccine,center_id,district_id_x,center_uuid,center_name,...,pincode,lat,lng,from_hour,to_hour,fee_type,district_id_y,district_id,district_name_y,state_id
0,51738255-efad-470e-8660-e9d829d83ccb,01-05-2021,01-05-2021,50.0,45,,570779,3,c7dad516-fa4c-4165-a731-3cf5692f7c42,BJR Hospital,...,744301,9.0,92.0,09:00:00,17:00:00,Free,3,3,Nicobar,1
1,51738255-efad-470e-8660-e9d829d83ccb,01-05-2021,01-05-2021,50.0,45,,570779,3,4474777c-9750-463e-bb80-f9ec292c1fba,BJR Hospital,...,744301,0.0,0.0,09:00:00,17:00:00,Free,3,3,Nicobar,1
2,51738255-efad-470e-8660-e9d829d83ccb,01-05-2021,01-05-2021,50.0,45,,570779,3,ab709a39-bb18-4a59-8525-e6a0bf28275e,BJR Hospital,...,744301,0.0,0.0,09:00:00,17:00:00,Free,3,3,Nicobar,1
3,51738255-efad-470e-8660-e9d829d83ccb,01-05-2021,01-05-2021,50.0,45,,570779,3,fe7f66d0-695e-4df7-a1d2-28be69188dd6,BJR Hospital,...,744301,0.0,0.0,09:00:00,17:00:00,Free,3,3,Nicobar,1
4,51738255-efad-470e-8660-e9d829d83ccb,01-05-2021,01-05-2021,50.0,45,,570779,3,f478ea34-7306-4444-ad3f-b712c275cd23,BJR Hospital,...,744301,0.0,0.0,09:00:00,17:00:00,Free,3,3,Nicobar,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1277648,e91c6b85-98a2-4757-ab94-bd7da8756e16,05-05-2021,05-05-2021,9.0,45,,551644,139,ee24752c-9afe-4cac-bb29-3a49f719573a,PHC Vanakbara,...,362540,20.0,70.0,09:00:00,17:00:00,Free,139,139,Diu,37
1277649,e91c6b85-98a2-4757-ab94-bd7da8756e16,05-05-2021,05-05-2021,9.0,45,,551644,139,e0440914-ea7a-4697-abbb-13b0bf7820ce,PHC Vanakbara,...,362540,20.0,70.0,09:00:00,17:00:00,Free,139,139,Diu,37
1277650,e91c6b85-98a2-4757-ab94-bd7da8756e16,05-05-2021,05-05-2021,9.0,45,,551644,139,2237f8a1-9e00-435a-a2ad-70cf5ede532b,PHC Vanakbara,...,362540,20.0,70.0,09:00:00,17:00:00,Free,139,139,Diu,37
1277651,e91c6b85-98a2-4757-ab94-bd7da8756e16,05-05-2021,05-05-2021,9.0,45,,551644,139,8b08cc10-d1c5-4d20-920c-b4b15e34f33f,PHC Vanakbara,...,362540,20.0,70.0,09:00:00,17:00:00,Free,139,139,Diu,37


In [263]:
# Top 5 States Running ahead of everyone else.
session_center_district_df[['date', 'state_name']].groupby('state_name').apply(lambda x : x.sort_values(by = 'date', ascending = True).head(3).reset_index(drop = True))

date                   state_name
state_name                                                            
Andaman and Nicobar Islands 0  01-05-2021  Andaman and Nicobar Islands
                            1  01-05-2021  Andaman and Nicobar Islands
                            2  01-05-2021  Andaman and Nicobar Islands
Andhra Pradesh              0  01-05-2021               Andhra Pradesh
                            1  01-05-2021               Andhra Pradesh
...                                   ...                          ...
Uttarakhand                 1  01-05-2021                  Uttarakhand
                            2  01-05-2021                  Uttarakhand
West Bengal                 0  01-05-2021                  West Bengal
                            1  01-05-2021                  West Bengal
                            2  01-05-2021                  West Bengal

[111 rows x 2 columns]

In [264]:
session_center_district_df\
    [['date', 'state_name']]\
    .groupby('state_name')\
    .apply(lambda x : x.sort_values(by = 'date', ascending = True)\
    .head(1)\
    .reset_index(drop = True)).head(40)

,,date,state_name
state_name,,,
Andaman and Nicobar Islands,0,01-05-2021,Andaman and Nicobar Islands
Andhra Pradesh,0,01-05-2021,Andhra Pradesh
Arunachal Pradesh,0,01-05-2021,Arunachal Pradesh
Assam,0,01-05-2021,Assam
Bihar,0,01-05-2021,Bihar
Chandigarh,0,01-05-2021,Chandigarh
Chhattisgarh,0,01-05-2021,Chhattisgarh
Dadra and Nagar Haveli,0,01-05-2021,Dadra and Nagar Haveli
Daman and Diu,0,01-05-2021,Daman and Diu


In [265]:
session_center_district_df\
    [session_center_district_df["min_age_limit"]<45]\
    [session_center_district_df["fee_type"]=="Free"]\
    [['date', 'state_name']]\
    .groupby('state_name')\
    .apply(lambda x : x.sort_values(by = 'date', ascending = True)\
    .head(1)\
    .reset_index(drop = True)).head(40)

<ipython-input-265-2df4eb2f42ac>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  session_center_district_df\


,,date,state_name
state_name,,,
Andhra Pradesh,0,01-05-2021,Andhra Pradesh
Assam,0,01-05-2021,Assam
Bihar,0,01-05-2021,Bihar
Chhattisgarh,0,01-05-2021,Chhattisgarh
Delhi,0,08-05-2021,Delhi
Gujarat,0,01-05-2021,Gujarat
Haryana,0,01-05-2021,Haryana
Jammu and Kashmir,0,01-05-2021,Jammu and Kashmir
Jharkhand,0,01-05-2021,Jharkhand


In [276]:
session_center_district_df.drop_duplicates(["date","state_id"]).groupby("state_id").apply(lambda x : x.sort_values(by = 'date', ascending = True).head(1).reset_index(drop = True)).head(40)

,,session_uuid,date,query_date,available_capacity,min_age_limit,vaccine,center_id,district_id_x,center_uuid,center_name,...,pincode,lat,lng,from_hour,to_hour,fee_type,district_id_y,district_id,district_name_y,state_id
state_id,,,,,,,,,,,,,,,,,,,,,,
1,0,51738255-efad-470e-8660-e9d829d83ccb,01-05-2021,01-05-2021,50.0,45,,570779,3,c7dad516-fa4c-4165-a731-3cf5692f7c42,BJR Hospital,...,744301,9.0,92.0,09:00:00,17:00:00,Free,3,3,Nicobar,1
2,0,4d4c4f9a-dbc3-437f-85ac-4ad12b5ad01c,01-05-2021,01-05-2021,45.0,45,,342882,9,bcd4bd5b-fe41-4268-8158-a6d91d08b537,Bhagyanagar EUPHC,...,515801,15.0,77.0,09:00:00,18:00:00,Free,9,9,Anantapur,2
3,0,aef58de3-7aaf-41a4-afde-386bb6cbc5c3,01-05-2021,01-05-2021,20.0,45,,105352,20,7d905ca7-82e0-4f4c-b811-77d5724f3021,Diyun CHC,...,792103,27.0,96.0,09:00:00,17:00:00,Free,20,20,Changlang,3
4,0,3a17b822-b241-4a44-b431-952bc7dcbaa8,01-05-2021,01-05-2021,48.0,45,,561639,46,220d29b6-4d3b-47fd-b78a-806726fe1bd8,Barama PHC,...,781346,26.0,91.0,09:00:00,18:00:00,Free,46,46,Baksa,4
5,0,ed9d5965-11fa-4966-aecb-69a0826052c1,01-05-2021,01-05-2021,50.0,45,,615264,74,93612f99-f04c-4d20-ac5a-55150ca9133b,APHC-HWC Diyari,...,854311,26.0,87.0,09:00:00,18:00:00,Free,74,74,Araria,5
6,0,1b0ce2f3-1884-47ce-be9e-47ebe1b7441c,01-05-2021,01-05-2021,19.0,45,,582765,108,1a3f4da6-3ad3-477a-8ea1-034c28ca768a,ESI Hospital Ramdarbar CHD,...,160002,30.0,76.0,09:00:00,15:00:00,Free,108,108,Chandigarh,6
7,0,3f9c9c61-a368-4185-a7d6-546bb7df4609,01-05-2021,01-05-2021,50.0,45,,584635,110,8854791e-4731-49cf-b222-8f423c62bcd1,Palari PHC 1,...,491222,0.0,0.0,09:00:00,18:00:00,Free,110,110,Balod,7
8,0,881b3e8d-dedd-4ba4-bfed-ae57d849ba1e,01-05-2021,01-05-2021,0.0,45,,365409,137,fb55077d-55c0-4ff9-b729-b95d945bc6d1,HWC Vasona,...,396230,20.0,73.0,09:00:00,17:00:00,Free,137,137,Dadra and Nagar Haveli,8
9,0,ff0fc23f-7e23-4eb7-82bf-5df84961c23d,01-05-2021,01-05-2021,198.0,45,,580638,141,40ee15bf-3b63-4a43-8f14-539b34877e76,ARUNA ASAF HOSPITAL SITE 2,...,110006,28.0,77.0,09:00:00,21:00:00,Free,141,141,Central Delhi,9


In [ ]:
session_center_district_df